In [ ]:
from util.commons import *
from MSSE-2021 import

In [7]:
train_dataloader, valid_dataloader, test_dataloader = get_dataloaders_dist(
    pre_processed_dir="/niddk-data-central/iWatch/pre_processed_pt",
    bi_lstm_win_size=10,
    batch_size=2,
    train_subjects=None,
    valid_subjects=None,
    test_subjects=None,
    rank=1,
    world_size=1,
)


In [1]:
import h5py

# Path to your HDF5 file
file_path = '/niddk-data-central/iWatch/pre_processed_pt/H/i0001A/2013-05-09.h5'

# Open the file in read-only mode and display its structure
with h5py.File(file_path, 'r') as f:
    print("Top-level groups and datasets:")
    for name, obj in f.items():
        print(f"  {name}: {obj.__class__.__name__}")
    
    print("\nDetailed dataset info:")
    def print_info(name, obj):
        if isinstance(obj, h5py.Dataset):
            print(f"  Dataset: {name}")
            print(f"    Shape: {obj.shape}")
            print(f"    Dtype: {obj.dtype}")
    f.visititems(print_info)


Top-level groups and datasets:
  data: Dataset
  label: Dataset
  non_wear: Dataset
  sleeping: Dataset
  time: Dataset

Detailed dataset info:
  Dataset: data
    Shape: (8640, 100, 3)
    Dtype: float64
  Dataset: label
    Shape: (8640,)
    Dtype: int64
  Dataset: non_wear
    Shape: (8640,)
    Dtype: int64
  Dataset: sleeping
    Shape: (8640,)
    Dtype: int64
  Dataset: time
    Shape: (8640,)
    Dtype: float64


In [2]:
f

<Closed HDF5 file>

In [14]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix
from torchmetrics.classification import BinaryAccuracy, MulticlassRecall, MulticlassSpecificity,MulticlassF1Score
# from torchmetrics import F1Score
import sys
sys.path.append('/Users/leo/Desktop/DeepPostures_MAE/MSSE-2021-pt/')
from utils import compute_additional_metrics_from_confusion_matrix, compute_accuracy_from_confusion_matrix

# ---- Simulate batch-wise imbalanced binary data with both FP and FN ----
batch1_true = torch.tensor([0]*45 + [1]*5)
batch1_pred = torch.tensor([0]*38 + [1]*7 + [0]*5)  # 5 TP, 5 FN, 7 FP

batch2_true = torch.tensor([0]*50 + [1]*5)
batch2_pred = torch.tensor([0]*43 + [1]*7 + [0]*5)  # 5 TP, 5 FN, 7 FP

# Combine batches
y_true = torch.cat([batch1_true, batch2_true])
y_pred = torch.cat([batch1_pred, batch2_pred])

# ---- Compute confusion matrix ----
cm = confusion_matrix(y_true.numpy(), y_pred.numpy(), labels=[0, 1])

# ---- Compute metrics using your utility functions ----
acc_np, bal_acc_np = compute_accuracy_from_confusion_matrix(cm)
metrics_np = compute_additional_metrics_from_confusion_matrix(cm)
f1_np_class0 = metrics_np["f1_score"][0]
f1_np_class1 = metrics_np["f1_score"][1]
support_class0 = cm[0].sum()
support_class1 = cm[1].sum()
weighted_f1_np = (f1_np_class0 * support_class0 + f1_np_class1 * support_class1) / (support_class0 + support_class1)

# ---- Compute metrics using torchmetrics ----
# Initialize metrics
accuracy_metric = BinaryAccuracy()
recall_metric = MulticlassRecall(num_classes=2,
                                 average='weighted',
                                 zero_division=0)
specificity_metric  = MulticlassSpecificity(num_classes=2,
    average='weighted',
    zero_division=0
)
f1_metric = MulticlassF1Score(num_classes=2,
                           average='weighted',
                           zero_division=0)

# Update metrics with batch1
accuracy_metric.update(batch1_pred, batch1_true)
recall_metric.update(batch1_pred, batch1_true)
specificity_metric.update(batch1_pred, batch1_true)
f1_metric.update(batch1_pred, batch1_true)

# Update metrics with batch2
accuracy_metric.update(batch2_pred, batch2_true)
recall_metric.update(batch2_pred, batch2_true)
specificity_metric.update(batch2_pred, batch2_true)
f1_metric.update(batch2_pred, batch2_true)

# Compute metrics
accuracy_tm = accuracy_metric.compute().item()
recall_tm = recall_metric.compute().item()
specificity_tm = specificity_metric.compute().item()
balanced_acc_tm = (recall_tm + specificity_tm) / 2

f1_tm = f1_metric.compute().item()

# ---- Print comparison ----
print(f"Manual Accuracy:          {acc_np:.4f}, Torchmetrics: {accuracy_tm:.4f}")
print(f"Manual Balanced Accuracy: {bal_acc_np:.4f}, Torchmetrics: {balanced_acc_tm:.4f}")
print(f"Manual Weighted F1 Score: {weighted_f1_np:.4f}, Torchmetrics: {f1_tm:.4f}")


/Users/leo/Desktop/DeepPostures_MAE/MSSE-2021-pt/utils.py:86: RuntimeWarning: invalid value encountered in divide
  f1_score = 2 * ppv * sensitivity / (ppv + sensitivity)


Manual Accuracy:          0.7714, Torchmetrics: 0.7714
Manual Balanced Accuracy: 0.4263, Torchmetrics: 0.4263
Manual Weighted F1 Score: 0.7880, Torchmetrics: 0.7880
